In [3]:
from neo4j import GraphDatabase

class IMDBELTLoader:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def execute_query(self, query, params_dict={}):
        with self.driver.session(database="neo4j") as session:
            result = session.run(query, **params_dict)
            print(result.consume().counters)


file_list = [
"elt_cypher_queries/cleanup.cypher",
"elt_cypher_queries/load_genres.cypher",
"elt_cypher_queries/load_region_codes.cypher",
"elt_cypher_queries/load_language_codes.cypher",
"elt_cypher_queries/load_distribution_types.cypher",
"elt_cypher_queries/load_title_types.cypher",
"elt_cypher_queries/load_job_categories.cypher",
"elt_cypher_queries/load_name_basics_node.cypher",
"elt_cypher_queries/load_attributes_node.cypher",
"elt_cypher_queries/load_title_akas_cleansed_node.cypher",
"elt_cypher_queries/load_title_basics_node.cypher",
"elt_cypher_queries/load_title_episode_node.cypher",
"elt_cypher_queries/load_title_principals_cleansed_node.cypher",
"elt_cypher_queries/load_title_ratings_node.cypher"
]


if __name__ == "__main__":
    for query_file in file_list:
        print(f"Processing {query_file}")
        with open(query_file) as f:
            query = f.read()
           
        loader = IMDBELTLoader("bolt://localhost:7687", "neo4j", "pass1234")
        loader.execute_query(query)
        loader.close()

Processing elt_cypher_queries/cleanup.cypher
{}
Processing elt_cypher_queries/load_genres.cypher
{'_contains_updates': True, 'labels_added': 28, 'nodes_created': 28, 'properties_set': 28}
Processing elt_cypher_queries/load_region_codes.cypher
{'_contains_updates': True, 'labels_added': 249, 'nodes_created': 249, 'properties_set': 498}
Processing elt_cypher_queries/load_language_codes.cypher
{'_contains_updates': True, 'labels_added': 184, 'nodes_created': 184, 'properties_set': 368}
Processing elt_cypher_queries/load_distribution_types.cypher
{'_contains_updates': True, 'labels_added': 8, 'nodes_created': 8, 'properties_set': 8}
Processing elt_cypher_queries/load_title_types.cypher
{'_contains_updates': True, 'labels_added': 11, 'nodes_created': 11, 'properties_set': 11}
Processing elt_cypher_queries/load_job_categories.cypher
{'_contains_updates': True, 'labels_added': 12, 'nodes_created': 12, 'properties_set': 12}
Processing elt_cypher_queries/load_name_basics_node.cypher
{'_contains